In [2]:
import matplotlib as plt
import numpy as np
import seaborn as sns
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import squarify
import ast
pd.options.mode.chained_assignment = None

**Tarea 1 VD**

-Visualizacion marcas de autos

In [21]:
#-------------------Manipulacion de datos-------------------
path3 = "datos_generados2.csv"

df3 = pd.read_csv(path3)

#obtencion de autos en las finales por marca
y_mazda = [df3[str(year)].value_counts().get("mazda",0) for year in range(2005,2025)]
y_toyota = [df3[str(year)].value_counts().get("toyota",0) for year in range(2005,2025)]
y_honda = [df3[str(year)].value_counts().get("honda",0) for year in range(2005,2025)]

#suma de los autos totales en finales
total_mazda = np.sum(y_mazda)
total_toyota = np.sum(y_toyota)
total_honda = np.sum(y_honda)

#se crea un pequeño dataframe
data = {
    'Marca': ['Mazda', 'Honda', 'Toyota'],
    'Autos': [total_mazda, total_honda, total_toyota]
}

df4 = pd.DataFrame(data)

df4

#-------------------Manipulacion de datos-------------------

#-----------------------Visualizacion----------------------

fig = px.treemap(data_frame=df4, path=[px.Constant('WCOTY'),'Marca'], values='Autos',color='Autos',color_continuous_scale='Viridis')

fig.update_layout(title='Marcas con mas autos finalistas del WCOTY 2005-2024')

fig.show()

#-----------------------Visualizacion----------------------


Visualizacion autores

In [59]:


from colorama import Fore
import requests
from bs4 import BeautifulSoup

urls = ['https://www.filmaffinity.com/cl/movie-group.php?group-id=40&chv=list&orderby=pos&p=1','https://www.filmaffinity.com/cl/movie-group.php?group-id=231&orderby=pos&chv=list','https://www.filmaffinity.com/cl/movie-group.php?group-id=5&p=1&orderby=pos&chv=list']

column_headers = ['Title','Author','Rating','Year','Awards','Users who rated']

title = ''
author = ''
rating = ''
year = ''
awards = ''
users_who_rated = ''

dataframe = pd.DataFrame(columns=column_headers)
numero_url = 1
for url in urls:
    print("Estamos en la url numero " + str(numero_url))
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html')
    lista = soup.find_all('ul', class_='movie-group-movies')

    etiqueta_autor = soup.find('h1',id='main-title')
    autor = etiqueta_autor.text.replace('Grupo: Adaptaciones de ','').strip()

    pager = soup.find('div',class_='pager')

    if pager:
        links_paginas = pager.find_all('a')
        arreglo_paginas = []
        for hrefs in links_paginas[:-1]:
            pagina = hrefs.get('href')
            arreglo_paginas.append(pagina)

    else:
        arreglo_paginas = []
    numero_url+=1
    
    loops = 0
    
    while loops < len(arreglo_paginas) + 1:
        for li in lista[0].find_all('li'):
            #obtener pagina de la obra para extraer premios
            a = li.find_all('a')[0]
            href = a.get('href')
            page2 = requests.get(href)
            soup2=BeautifulSoup(page2.text,'html')
            #-----------------------------------------------------------

            author = autor
            for nombre_obra in li.find_all('div', class_='mc-title'):#titulo
                title = nombre_obra.text.strip()
            for anio in li.find_all('span',class_='mc-year'):#año
                year = anio.text.strip()
            for rate in li.find_all('div',class_='avgrat-box'):#rating
                rating = rate.text.strip()
            for count in li.find_all('div',class_='ratcount-box'):#usuarios que calificaron
                users_who_rated = count.text.strip()
            #obtener premios de la obra
            
            premios = soup2.find_all('dd',class_='award')
            if premios:
                etiqueta_em = premios[0].find('em')
                a2 = etiqueta_em.find('a')
                href2 = a2.get('href')
                
                awards_page = requests.get(href2)
                soup3 = BeautifulSoup(awards_page.text, 'html')

                ul_premios = soup3.find('ul',class_= 'awards-list')

                lista_premios = []
                for li_premios in ul_premios.find_all('li'):
                    for ul_premio in li_premios.find_all('ul'):
                        for li_premio in ul_premio.find_all('li'):
                            premio = li_premio.find('a').text.strip()
                            lista_premios.append(premio)
                awards = lista_premios
                fila_temp = pd.Series([title, author, rating, year, awards, users_who_rated], index=column_headers)
                dataframe = pd.concat([dataframe, fila_temp.to_frame().T], ignore_index=True)

            else:
                awards = ""
                fila_temp = pd.Series([title, author, rating, year, awards, users_who_rated], index=column_headers)
                dataframe = pd.concat([dataframe, fila_temp.to_frame().T], ignore_index=True)  
        loops+=1
        if (len(arreglo_paginas) >= 1) and (loops <= len(arreglo_paginas)):
            page = requests.get(arreglo_paginas[loops - 1])
            soup = BeautifulSoup(page.text,'html')
            lista = soup.find_all('ul', class_='movie-group-movies')



dataframe





Estamos en la url numero 1
Estamos en la url numero 2
Estamos en la url numero 3


,Title,Author,Rating,Year,Awards,Users who rated
0,Carrie,Stephen King,"7,0",1976,"[Mejor actriz principal, Mejor actriz de repar...",57.306
1,Phantasma II,Stephen King,"6,2",1979,,703
2,El misterio de Salem's Lot,Stephen King,"6,4",1979,,3.361
3,El resplandor,Stephen King,"8,2",1980,"[Peor dirección, Peor actriz, Mejor productor ...",149.166
4,El hombre del saco,Stephen King,"4,4",1982,,116
...,...,...,...,...,...,...
174,El Señor de los Anillos: La Guerra del Norte,J. R. R. Tolkien,"6,0",2011,,25
175,The Lord of the Rings: Tactics,J. R. R. Tolkien,--,2005,,25
176,El Hobbit,J. R. R. Tolkien,--,2003,,25
177,Tales of the Shire,J. R. R. Tolkien,--,2024,,25


Parte de escritores


In [22]:
#dataframe.to_csv('filmaffinity.csv', index=False)
#df_sin_duplicados = dataframe.drop_duplicates(subset=[col for col in dataframe.columns if col != 'Awards'])
#df_sin_duplicados.to_csv('sinDuplicados.csv', index=False)

#-------------------Limpieza y orden de los datos-------------------
path = 'filmaffinity.csv'
df = pd.read_csv(path)

#eliminar duplicados
df_sin_duplicados = df.drop_duplicates(subset=[col for col in df.columns if col != 'Awards']) 

#crear un nuevo dataframe con la informacion que vamos a manipular
nuevo_dataframe = df_sin_duplicados.loc[:, ['Author', 'Awards']]

#se eliminan los espacios vacios
dataframe_sin_vacios = nuevo_dataframe.dropna(subset=['Awards'], thresh=1) 

#se reemplazan los espacios vacios por una cadena cualquiera
dataframe_sin_vacios['Awards'] = dataframe_sin_vacios['Awards'].replace("","buenas")

#se eliminan las "" que encierran a las listas en el csv
dataframe_sin_vacios['Awards'] = dataframe_sin_vacios['Awards'].apply(ast.literal_eval)

#se filtra el dataframe ahora sin las filas con valor buenas
dataframe_sin_vacios = dataframe_sin_vacios[dataframe_sin_vacios['Awards'] != "buenas"] 

# se filtrar el dataframe para excluir las filas que tienen listas vacías en la columna 'Awards'
dataframe_sin_vacios = dataframe_sin_vacios[dataframe_sin_vacios['Awards'].apply(lambda x: len(x) != 0)]

# se separan las listas de la columna Awards
df_final = dataframe_sin_vacios.explode('Awards')

#-------------------Limpieza y orden de los datos-------------------




#---------------------Visualizacion----------------------------

# se obtienen los 15 premios más repetidos
top_awards = df_final['Awards'].value_counts().head(15).index

# se filtra el dataframe para incluir solo los premios más repetidos
df_top_awards = df_final[df_final['Awards'].isin(top_awards)]

# se agrupa por autor y premio y contar la frecuencia
grouped = df_top_awards.groupby(['Author', 'Awards']).size().reset_index(name='Frequency')

# Crear listas de nodos y enlaces
nodos = []
nodo_labels = {}
nodo_indices = 0
links = []
colores_enlaces = ['#927785', '#74a7e4', '#eced87']
for author, award, freq in zip(grouped['Author'], grouped['Awards'], grouped['Frequency']):
    if author not in nodo_labels:
        nodo_labels[author] = nodo_indices
        nodos.append(author)
        nodo_indices += 1
    if award not in nodo_labels:
        nodo_labels[award] = nodo_indices
        nodos.append(award)
        nodo_indices += 1
    links.append({
        'source': nodo_labels[author],
        'target': nodo_labels[award],
        'value': freq,
        'color': colores_enlaces[nodo_labels[author] % len(colores_enlaces)]
    })


fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodos
    ),
    link=dict(
        source=[link['source'] for link in links],
        target=[link['target'] for link in links],
        value=[link['value'] for link in links],
        color=[link['color'] for link in links]
    )
)])

fig.update_layout(title_text="Adaptaciones de escritores nominadas/ganadoras de premios")

fig.show()

#---------------------Visualizacion----------------------------

